# Chapter4

## 신경망 학습
- 학습이란?
	- 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것을 뜻 합니다.


### 목표
	1. 장에서는 신경망이 학습 할 수 있도록 해주는 지표인 '손실 함수'를 소개하겠습니다.
	2. 신경망 학습(데이터로부터 매개변수의 값을 정하는 방법)에 대해서 설명하고 파이썬으로 MNIST 데이터셋의 손글씨 숫자를 학습하는 코드를 구현해보겠습니다.

#### 4.1.1 데이터 주도 학습
```
MNIST에서 해당 숫자들을 인식하는 방법은, 이미지에서 특징(Feature)을 추출하고 그 특징의 패턴을 기계학습 기술로 학습하는 방법이 있습니다. 
이런 특징을 사용하여 이미지 데이터를 벡터로 변환하고, 변환된 벡터를 가지고 지도 학습 방식의 대표 분류 기법인 SVM, KNN 등으로 학습할 수 있습니다.
다만, 이미지를 벡터로 변환할 때 사용하는 특징은 여전히 '사람'이 설계하는 것임을 주의해야 합니다.
```

- 이러한 문제를 해결하기 위해 딥러닝은 '종단간 기계학습(end-to-end machine learning')이라고도 합니다. 처음부터 끝까지 데이터(입력)에서 목표한 결과(출력)를 사람의 개입 없이 얻는다는 의미를 가지고 있습니다.

#### 4.1.2 훈련 데이터와 시험 데이터
- 두 데이터를 나눈 이유?
	- '범용 능력'을 제대로 평가하기 위함입니다. 범용능력은 아직 보지 못한 데이터로도 문제를 올바르게 풀어내는 능력입니다.
- 오버피팅(overfitting)이란?
	- 예를들어, 범용능력은 '특정인의 특정 손글씨'를 판별하는 것이아니라 '임의의 사람의 임의의 글자'를 올바르게 판별해야 합니다. 하지만, 한 데이터셋에만 지나치게 최적화된 교육이 이루어지면 '오버피팅'이 발생합니다.

#### 4.2 손실 함수
- 손실함수란?
	- 신경망 학습에서는 현재의 상태를 '하나의 지표'로 표현합니다. 그리고 그 지표를 가장 좋게 만들어주는 최적의 가중치 매개변수의 값을 탐색합니다. 이처럼 신경망 학습에서 사용하는 지표는 '손실 함수(loss function)'라고 합니다.
	- 일반적으로, 평균 제곱 오차(MSE : mean squared error)와 교차 엔트로피 오차를 사용합니다.
	- 손실 함수는 신경망 성능의 '나쁨'을 나타내는 지표입니다. 현재의 신경망이 훈련 데이터를 얼마나 잘 처리하지 못 하고 있는지를 나타내는 지표입니다.

#### 4.2.1 평균 제곱 오차(MSE : mean squared error)
-![MSE Function](https://i.stack.imgur.com/iSWyZ.png)
- yi는 신경망의 출력(신경망이 추정한 값), ti는 정답 레이블. i는 데이터의 차원 수를 나타냅니다.

In [23]:
import numpy as np

def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [24]:
# 정답 2
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # 원-핫 인코딩
# => 정답 레이블, 정답은 1 나머지는 0 

# 예1
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] # 소프트 맥스 함수
# => 0일 확률 0.1, 1일 확률 0.05 ... 의미

mean_squared_error(np.array(y), np.array(t))

0.097500000000000031

In [25]:
# 예2 : '7'일 확률이 가장 높다고 추정함(0.6)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0] # 소프트 맥스 함수
# => 0일 확률 0.1, 1일 확률 0.05 ... 의미

mean_squared_error(np.array(y), np.array(t))

0.59750000000000003

- 평균 제곱 오차 예제를 통한 내용 
    - 첫 번째 예의 손실 함수 쪽 출력이 작으면 정답 레이블과의 오차도 적은 것을 알 수 있습니다. 즉, 평균 제곱 오차를 기준으로는 첫 번째 추정 결과가(오차가 더 작은 값) 정답에 더 가까울 것으로 팓단 할 수 있습니다.


In [26]:
#### 4.2.2 교차 엔트로피 오차(CEE : cross entropy error)